In [1]:
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

words = []
classes = []
documents = []
ignore_words = ["?", "!"]
data_file = open("data.json").read()
intents = json.loads(data_file)


for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # add documents in the corpus
        documents.append((w, intent["tag"]))

        # add to our classes list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)


pickle.dump(words, open("texts.pkl", "wb"))
pickle.dump(classes, open("labels.pkl", "wb"))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

# fitting and saving the model
hist = model.fit(
    np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1
)
model.save("model.h5", hist)

print("model created")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']
Training data created


<ipython-input-1-d3e3aeaddab4>:77: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
10/10 [==============================] - 1s 4ms/step - loss: 2.2387 - accuracy: 0.0851
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1658 - accuracy: 0.1277
Epoch 3/200
10/10 [==============================] - 0s 4ms/step - loss: 2.0402 - accuracy: 0.2979
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 1.9202 - accuracy: 0.3830
Epoch 5/200
10/10 [==============================] - 0s 3ms/step - loss: 1.9089 - accuracy: 0.3191
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 1.7792 - accuracy: 0.4468
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 1.6338 - accuracy: 0.5106
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.4673 - accuracy: 0.5532
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.2933 - accuracy: 0.5957
Epoch 10/200
10/10 [==============================] - 0s 4ms/step - loss: 1.2823 - accuracy: 0.6596
Epoch 11/

In [3]:
!pip install pyngrok --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [11]:
from pyngrok import ngrok
ngrok.kill()
NGROK_AUTH_TOKEN = "2MSbbMnJBIeXVuNgRizrAkg9f5x_FM9xJtr38C3aMzHKgqkP"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [8]:
ngrok.connect(5000)

<NgrokTunnel: "http://2a81-34-80-38-242.ngrok.io" -> "http://localhost:5000">

In [9]:
import nltk

nltk.download("popular")
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model

model = load_model("model.h5")
import json
import random

intents = json.loads(open("data.json").read())
words = pickle.load(open("texts.pkl", "rb"))
classes = pickle.load(open("labels.pkl", "rb"))


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence


def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)


def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]["intent"]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result


def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


from flask import Flask, render_template, request

app = Flask(__name__)
app.static_folder = "static"


@app.route("/")
def home():
    return render_template("index.html")


@app.route("/get")
def get_bot_response():
    userText = request.args.get("msg")
    return chatbot_response(userText)


if __name__ == "__main__":
    app.run()


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [03/Mar/2023 13:07:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Mar/2023 13:07:24] "GET /static/styles/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Mar/2023 13:07:26] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 241ms/step


INFO:werkzeug:127.0.0.1 - - [03/Mar/2023 13:07:32] "GET /get?msg=hello HTTP/1.1" 200 -


...

INFO:werkzeug:127.0.0.1 - - [03/Mar/2023 13:16:12] "GET /get?msg=I%20have%20to%20complain%20regarding%20your%20services HTTP/1.1" 200 -


1/1 [==============================] - 0s 26ms/step


INFO:werkzeug:127.0.0.1 - - [03/Mar/2023 13:16:31] "GET /get?msg=Meenakshi%20Mission,%20Madurai HTTP/1.1" 200 -


1/1 [==============================] - 0s 30ms/step


INFO:werkzeug:127.0.0.1 - - [03/Mar/2023 13:16:41] "GET /get?msg=New%20York HTTP/1.1" 200 -


In [12]:
!ngrok.kill()

/bin/bash: -c: line 1: syntax error: unexpected end of file
